In [5]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Dump MYSQL") \
    .config("spark.jars", "/home/jovyan/work/mysql-connector-java-5.1.48.jar") \
    .getOrCreate()

table = 'tweets-2017-10'

def db():
    return spark.read \
        .format("jdbc") \
        .option("driver", 'com.mysql.jdbc.Driver') \
        .option("url", "jdbc:mysql://mysql/tweeter") \
        .option("user", "root") \
        .option("password", "")

query = f'(SELECT min(id), max(id) FROM `{table}`) AS tmp'

lower, upper = db() \
    .option("dbtable", query) \
    .load() \
    .first()

month = db() \
    .option("dbtable", f"`{table}`") \
    .option("partitionColumn", "id") \
    .option("numPartitions","4800") \
    .option("lowerBound",f"{lower}") \
    .option("upperBound",f"{upper}") \
    .load()


month.write.parquet(f'{table}.parquet')